## 在运行本笔记本之前，请先下载以下工具：

1. **查找当前的 Google Chrome 版本**  
   - 打开 Google Chrome，并在地址栏输入：  
     ```
     chrome://settings/help
     ```
   - 建议使用 **133** 版本的 Google Chrome（或最新可用的稳定版本）。

2. **下载对应版本的 ChromeDriver**  
   - 前往官方的 Chrome 测试下载页面：  
     [https://googlechromelabs.github.io/chrome-for-testing/#stable](https://googlechromelabs.github.io/chrome-for-testing/#stable)  
   - 下载与您的 Chrome 版本相匹配的 ChromeDriver。  
   - 例如，如果您的 Chrome 版本是 133，请下载 **ChromeDriver 133**。

3. **查找已下载的 ChromeDriver**  
   - 如果您使用 macOS，可在终端 (Terminal) 中运行以下命令来查找 `chromedriver` 的位置：
     ```bash
     mdfind -name chromedriver
     ```
   - 如果使用其他操作系统，请检查默认的 **下载** 目录或您保存该文件的目录。

In [16]:
import time
import random
import os
import requests
import pandas as pd
from tqdm import tqdm
from scrapy.selector import Selector
from scrapy.http import TextResponse
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

##### 启动 Chrome 浏览器实例：

打开 **terminal**, 下载 Chrome Driver (假的 Google Chrome)

```bash
brew install chromedriver
chmod +x /opt/homebrew/bin/chromedriver
```

输入以下命令（将 `your Chrome.exe path` 替换为您的 Google Chrome 浏览器路径）：
```bash
<your Chrome.exe path> --remote-debugging-port=9222 --user-data-dir="/Users/<your home folder name>/selenium/AutomationProfile"
```

- 请将your Chrome.exe path替换为您的Chrome浏览器所在路径，例如<br>`C:\Program Files\Google\Chrome\Application\chrome.exe`
- 配置 chromedriver 相关信息，请参考官方文档：[ChromeDriver](https://developer.chrome.com/docs/chromedriver)
- 来做个比方， 我的 *terminal command* 会是:

/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome \
  --remote-debugging-port=9222 \
  --user-data-dir="/Users/princess/selenium/AutomationProfile"

In [17]:
# 配置Chrome浏览器
options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')
options.add_argument('--incognito')
browser = webdriver.Chrome(options=options)
action = ActionChains(browser)

In [18]:
user_url = ""
num = 0

def check_login_status(browser):
    print("即将开始检查小红书登录状态...")
    print("爬取数据有账户封禁的风险，建议使用非主账号登录。")
    
    while True:
        page_source = browser.page_source
        if '登录探索更多内容' in page_source:
            print('暂未登录，请手动登录')
            print('检查时间:', time.ctime())
            time.sleep(10)
        else:
            print('登录成功')
            print('检查时间:', time.ctime())
            time.sleep(3)
            break

def check_user_page_load_status(browser):
    """
    检查用户主页是否加载成功。
    这里等待页面中出现至少一个帖子（包含 "note-item" 的 section）。
    """
    print("检查用户主页加载状态...")
    try:
        wait = WebDriverWait(browser, 15)
        wait.until(EC.presence_of_element_located((By.XPATH, '//section[contains(@class, "note-item")]')))
        print("用户主页加载成功")
    except Exception as e:
        print("用户主页加载超时或出错:", e)

def selenium_test():
    """
    登录状态检查、页面加载检查，并根据用户输入跳转到指定的用户主页进行爬取
    """
    global user_url, num
    # 首先打开探索页以触发登录检查
    browser.get('https://www.xiaohongshu.com/explore')
    check_login_status(browser)
    
    print("请根据提示输入用户主页URL和笔记爬取数量。")
    user_url = input("用户主页URL：").strip()
    try:
        num = int(input("笔记爬取数量："))
    except ValueError:
        print("请输入有效的整数作为爬取数量。")
        return
    
    # 跳转到指定用户主页
    browser.get(user_url)
    time.sleep(3)
    check_user_page_load_status(browser)

# 调用 selenium_test() 进行初始化（确保此处 browser 已创建）
selenium_test()

即将开始检查小红书登录状态...
爬取数据有账户封禁的风险，建议使用非主账号登录。
登录成功
检查时间: Thu Feb  6 18:43:21 2025
请根据提示输入用户主页URL和笔记爬取数量。
检查用户主页加载状态...
用户主页加载成功


In [19]:
# 初始化数据存储列表
authorName_list = []
likeNr_list = []
URL_list = []
userURL_list = []
commentNr_list = []
post_title_list = [] 
caption_list = []  # 存储帖子内容
datePublished_list = []
images_list = []
author_avatar_list = []  # 用于存储头像图片
starNr_list = []
authorCollectNr_list = []
authorFansNr_list = []
authorNoteNr_list = []
video_urls = [] 

def parsePage(page_source):
    """
    解析当前页面的HTML内容，提取用户笔记的基本信息并更新对应的列表。
    
    Args:
        page_source (str): 当前页面的HTML内容
    """
    response = TextResponse(url=browser.current_url, body=page_source.encode('utf-8'), encoding='utf-8')
    selector = Selector(response)

    print("正在分析页面结构...")

    content_elements = selector.xpath('//section[contains(@class, "note-item")]')
    if content_elements:
        print(f"找到 {len(content_elements)} 个内容元素")

        for element in content_elements:
            try:
                # 提取帖子（笔记）的相对URL
                note_url = element.xpath('.//a[contains(@class, "cover")]/@href').get()
                if note_url:
                    note_id = note_url.split('/')[-1].split('?')[0]
                    if note_id in URL_list:
                        continue  # 避免重复爬取
                    URL_list.append(note_id)
                    userURL_list.append(note_url)

                    # 提取作者名字（通常与用户主页相同）
                    author = element.xpath('.//div[contains(@class, "author-wrapper")]//span[contains(@class, "name")]/text()').get()
                    authorName_list.append(author.strip() if author else "N/A")

                    # 提取点赞数量
                    likes = element.xpath('.//span[contains(@class, "like-wrapper")]/span[contains(@class, "count")]/text()').get()
                    likeNr_list.append(likes.strip() if likes else "0")

                    # 提取帖子标题
                    post_title = element.xpath('.//a[contains(@class, "title")]//span/text()').getall()
                    post_title_cleaned = ' '.join([c.strip() for c in post_title if c.strip()])
                    post_title_list.append(post_title_cleaned if post_title_cleaned else "N/A")

                    # 提取图片（主图）
                    main_image = element.xpath('.//a[contains(@class, "cover")]/img/@src').get()
                    images_list.append(main_image.strip() if main_image else "N/A")

                    # 提取头像图片
                    avatar_image = element.xpath('.//a[contains(@class, "author")]/img/@src').get()
                    author_avatar_list.append(avatar_image.strip() if avatar_image else "N/A")

                    # 初始化附加字段的默认值
                    commentNr_list.append("0")
                    datePublished_list.append("N/A")
                    starNr_list.append("0")
                    authorCollectNr_list.append("0")
                    authorFansNr_list.append("0")
                    authorNoteNr_list.append("0")
                    video_urls.append("N/A")
                    caption_list.append("N/A")

                    qbar.update(1)

            except Exception as e:
                print(f"处理元素时出错: {str(e)}")
                continue

    print(f"当前已爬取总数: {len(URL_list)}")

def extract_additional_fields(note_url, note_id):
    """
    访问每个笔记的页面，提取附加字段，包括评论数量、发布时间、收藏数量、粉丝数量、笔记数量、视频URL以及帖子内容（Caption）。
    
    Args:
        note_url (str): 笔记的相对URL
        note_id (str): 笔记的唯一ID
    """
    try:
        full_note_url = f'https://www.xiaohongshu.com{note_url}'
        browser.get(full_note_url)
        
        # 等待页面加载中关键的描述 meta 标签
        wait = WebDriverWait(browser, 15)
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@name="description"]')))
        
        page_source = browser.page_source
        response = TextResponse(url=browser.current_url, body=page_source.encode('utf-8'), encoding='utf-8')
        selector = Selector(response)

        # 提取评论数量
        comments = selector.xpath('//*[@class="total"]/text()').get()
        comments = comments.strip() if comments else "0"

        # 提取发布时间（采用多种 XPath 尝试）
        date_published = selector.xpath('//*[@class="date"]/text()').get()
        if not date_published:
            date_published = selector.xpath('//time/@datetime').get()
        date_published = date_published.strip() if date_published else "N/A"

        # 提取收藏数量
        stars = selector.xpath('//*[@class="count"]/text()').get()
        stars = stars.strip() if stars else "0"

        # 提取作者收藏数量
        collect_nr = selector.xpath('//span[contains(@class, "collect") or contains(@class, "saved")]/text()').get()
        collect_nr = collect_nr.strip() if collect_nr else "0"

        # 提取作者粉丝数量
        fans_nr = selector.xpath('//span[contains(@class, "fans") or contains(@class, "followers")]/text()').get()
        fans_nr = fans_nr.strip() if fans_nr else "0"

        # 提取作者笔记数量
        note_nr = selector.xpath('//span[contains(@class, "notes") or contains(@class, "posts")]/text()').get()
        note_nr = note_nr.strip() if note_nr else "0"

        # 提取视频 URL（如果有）
        video_url = selector.xpath('//video/@src').get()
        video_url = video_url.strip() if video_url else "N/A"

        # 提取帖子内容（Caption）
        caption = selector.xpath('//*[@name="description"]/@content').get()
        caption = caption.strip() if caption else "N/A"

        # 更新全局列表中的相应条目
        if note_id in URL_list:
            index = URL_list.index(note_id)
            commentNr_list[index] = comments
            datePublished_list[index] = date_published
            starNr_list[index] = stars
            authorCollectNr_list[index] = collect_nr
            authorFansNr_list[index] = fans_nr
            authorNoteNr_list[index] = note_nr
            video_urls[index] = video_url
            caption_list[index] = caption
            print(f"已提取附加字段，笔记ID: {note_id}")
        else:
            print(f"笔记ID {note_id} 未在 URL_list 中找到。")

    except Exception as e:
        print(f"提取附加字段时出错，笔记ID: {note_id}, 错误: {str(e)}")

def download_image(url, save_path):
    """
    下载图片并保存到指定路径。
    
    Args:
        url (str): 图片的 URL 地址
        save_path (str): 图片保存的本地路径
    
    Returns:
        bool: 下载是否成功
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    except Exception as e:
        print(f"下载图片时出错，URL: {url}, 错误: {str(e)}")
        return False

def create_directories():
    """
    创建用于存储主图片和头像图片的目录。
    """
    if not os.path.exists('images_user'):
        os.makedirs('images_user')
    if not os.path.exists('avatars_user'):
        os.makedirs('avatars_user')

# 创建目录以存储图片
create_directories()

# 定义进度条用于跟踪已爬取的笔记数量
qbar = tqdm(total=num, desc="爬取进度")

# 开始下拉加载页面直至达到所需笔记数量
while len(URL_list) < num:
    for _ in range(3):
        browser.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)

    parsePage(browser.page_source)

    # 检测是否到达页面末尾（根据页面提示文本判断）
    if '- THE END -' in browser.page_source or 'No more content' in browser.page_source:
        print(f"已到达内容末尾。总共收集: {len(URL_list)} 条")
        break

    time.sleep(random.uniform(2, 4))

print(f"总共收集的条目数: {len(URL_list)}")
if URL_list:
    print("收集的数据样本:")
    for i in range(min(5, len(URL_list))):
        print(f"URL: {URL_list[i]}")

# 如果收集的条目超过用户所需数量，则截断各个列表
if len(URL_list) > num:
    URL_list = URL_list[:num]
    authorName_list = authorName_list[:num]
    likeNr_list = likeNr_list[:num]
    userURL_list = userURL_list[:num]
    commentNr_list = commentNr_list[:num]
    post_title_list = post_title_list[:num]
    datePublished_list = datePublished_list[:num]
    images_list = images_list[:num]
    author_avatar_list = author_avatar_list[:num]
    starNr_list = starNr_list[:num]
    authorCollectNr_list = authorCollectNr_list[:num]
    authorFansNr_list = authorFansNr_list[:num]
    authorNoteNr_list = authorNoteNr_list[:num]
    video_urls = video_urls[:num]
    caption_list = caption_list[:num]

print(f"截断后的总条目数: {len(URL_list)}")
print("收集的数据样本:")
for i in range(min(5, len(URL_list))):
    print(f"作者: {authorName_list[i]}, 点赞: {likeNr_list[i]}, URL: {URL_list[i]}")

qbar.close()

# 提取每个笔记的附加字段（例如帖子内容、发布时间和评论数量）
print("开始提取附加字段，包括帖子内容、日期发布和评论数量...")
qbar = tqdm(total=len(URL_list), desc="已获取的笔记数量...")

for note_id, note_url in zip(URL_list, userURL_list):
    extract_additional_fields(note_url, note_id)
    qbar.update(1)
    time.sleep(random.uniform(2, 4))  # 礼貌等待，避免服务器过载

qbar.close()

# 下载主图片和头像图片
print("开始下载主图片和头像图片...")

# 下载主图片
image_download_bar = tqdm(total=len(images_list), desc="下载主图片")
for idx, image_url in enumerate(images_list):
    if image_url == "N/A":
        image_download_bar.update(1)
        continue
    # 构造图片保存路径，使用 note_id 作为文件名
    image_extension = os.path.splitext(image_url)[1].split('?')[0]
    if image_extension.lower() not in ['.jpg', '.jpeg', '.png', '.gif']:
        image_extension = '.jpg'
    image_filename = f"images_user/{URL_list[idx]}{image_extension}"
    success = download_image(image_url, image_filename)
    if not success:
        image_filename = "N/A"
    images_list[idx] = image_filename
    image_download_bar.update(1)
image_download_bar.close()

# 下载头像图片
avatar_download_bar = tqdm(total=len(author_avatar_list), desc="下载头像图片")
for idx, avatar_url in enumerate(author_avatar_list):
    if avatar_url == "N/A":
        avatar_download_bar.update(1)
        continue
    avatar_extension = os.path.splitext(avatar_url)[1].split('?')[0]
    if avatar_extension.lower() not in ['.jpg', '.jpeg', '.png', '.gif']:
        avatar_extension = '.jpg'
    avatar_filename = f"avatars_user/{URL_list[idx]}{avatar_extension}"
    success = download_image(avatar_url, avatar_filename)
    if not success:
        avatar_filename = "N/A"
    author_avatar_list[idx] = avatar_filename
    avatar_download_bar.update(1)
avatar_download_bar.close()

print("所有图片下载完成。")

# 将数据整理为字典（包括“Author Avatar”与“Caption”字段）
data = {
    'Author Name': authorName_list,
    'Likes': likeNr_list,
    'Comments': commentNr_list,
    'Post Title': post_title_list, 
    'Caption': caption_list,
    'Date Published': datePublished_list,
    'Images': images_list,          # 主图片的本地路径
    'Author Avatar': author_avatar_list,  # 头像图片的本地路径
    'Stars': starNr_list,
    'Author Collect Nr': authorCollectNr_list,
    'Author Fans Nr': authorFansNr_list,
    'Author Note Nr': authorNoteNr_list,
    'Video URL': video_urls,
    'URL': URL_list,
    'User URL': userURL_list
}

# 将数据存储为 DataFrame 并导出 CSV
df = pd.DataFrame(data)
df.set_index('URL', inplace=True)
print(df.head())
df.to_csv('scraped_xhs_user.csv', encoding='utf-8-sig')
print("数据已保存到 'scraped_xhs_user.csv'")

爬取进度:  20%|██        | 1/5 [00:03<00:12,  3.08s/it]

正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21


爬取进度: 21it [00:06,  3.29it/s]                      


总共收集的条目数: 21
收集的数据样本:
URL: 676f71ba000000000b0227ac
URL: 63158aee0000000011036e6e
URL: 62724ac20000000001027847
URL: 62639a4100000000010294ec
URL: 625a9529000000002103e554
截断后的总条目数: 5
收集的数据样本:
作者: 黄子韬, 点赞: 1.5万, URL: 676f71ba000000000b0227ac
作者: 黄子韬, 点赞: 5620, URL: 63158aee0000000011036e6e
作者: 黄子韬, 点赞: 1万, URL: 62724ac20000000001027847
作者: 黄子韬, 点赞: 1万, URL: 62639a4100000000010294ec
作者: 黄子韬, 点赞: 2.4万, URL: 625a9529000000002103e554
开始提取附加字段，包括帖子内容、日期发布和评论数量...


已获取的笔记数量...:  20%|██        | 1/5 [00:05<00:22,  5.53s/it]

...

已提取附加字段，笔记ID: 625a9529000000002103e554


已获取的笔记数量...: 100%|██████████| 5/5 [00:39<00:00,  7.99s/it]


开始下载主图片和头像图片...


下载头像图片: 100%|██████████| 5/5 [00:00<00:00, 12.06it/s]

所有图片下载完成。
                         Author Name Likes    Comments  \
URL                                                      
676f71ba000000000b0227ac         黄子韬  1.5万  共 2094 条评论   
63158aee0000000011036e6e         黄子韬  5620  共 1228 条评论   
62724ac20000000001027847         黄子韬    1万           0   
62639a4100000000010294ec         黄子韬    1万  共 1431 条评论   
625a9529000000002103e554         黄子韬  2.4万  共 3317 条评论   

                                                        Post Title  \
URL                                                                  
676f71ba000000000b0227ac  谁熬夜，call me ，我用绝色宠你！很高兴成为@绝色JIASEE 品牌全球代   
63158aee0000000011036e6e                                    新款即将降世   
62724ac20000000001027847                                         🎥   
62639a4100000000010294ec                                打工👨🏻‍🏭OOTD   
625a9529000000002103e554                               好久不见 🍠的兄弟们！   

                                                                    Caption  \
URL          